In [49]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn 
import torch.nn.functional as F

from sklearn.metrics import balanced_accuracy_score

### Data Pre-Processing
---

In [50]:
def train_test_split(df):
    msk = np.random.rand(len(df)) < 0.9
    train = df[msk].reset_index()
    valid = df[~msk].reset_index()
    return train.drop(columns=['index']), valid.drop(columns=['index'])

In [51]:
def init_model_and_data(train):
    train_df = encode_data(train, train=None)
    num_users = len(train.user_id.unique())
    num_items = len(train.item_id.unique())
    model = MF(num_users, num_items)
    return model, train_df

In [52]:
def proc_col(col, train_col=None):
    """Encodes a pandas column with continous ids. 
    """
    if train_col is not None:
        uniq = train_col.unique()
    else:
        uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx.get(x, -1) for x in col]), len(uniq)

In [53]:
def encode_data(df, train=None):
    """ Encodes rating data with continous user and movie ids. 
    If train is provided, encodes df with the same encoding as train.
    """
    df = df.copy()
    for col_name in ["user_id", "item_id"]:
        train_col = None
        if train is not None:
            train_col = train[col_name]
        _,col,_ = proc_col(df[col_name], train_col)
        df[col_name] = col
        df = df[df[col_name] >= 0]
    return df

In [54]:
def encode_item_features(df, train=None):
    """ Encodes rating data with continous user and movie ids. 
    If train is provided, encodes df with the same encoding as train.
    """
    df = df.copy()
    for col_name in ["item_feature_id"]:
        train_col = None
        if train is not None:
            train_col = train[col_name]
        _,col,_ = proc_col(df[col_name], train_col)
        df[col_name] = col
        df = df[df[col_name] >= 0]
    return df

In [55]:
def add_samples(df, k, all_users, all_items, user_context_features):
    sample_df = {'user_id':np.empty(df.shape[0]*k),
        'item_id':np.empty(df.shape[0]*k),
        'rating':np.empty(df.shape[0]*k),
        'context_feature_id':np.empty(df.shape[0]*k)}
    for user in all_users:
        split_index = np.where(df[:,0] == user)[0]
        user_df = df[split_index[0]:split_index[-1]+1]
        
        zero_samples = sample_zeros(user_df, k, all_items)
        user_col = np.full(user_df.shape[0]*k, user)
        zero_col = np.zeros(user_df.shape[0]*k)

        context_features = np.random.choice(user_context_features[user_df[0,0]],
         user_df.shape[0]*k)

        # sample_df = {'user_id':user_col, 'item_id':zero_samples, 'rating':zero_col}
        sample_df['user_id'][split_index[0]:split_index[-1]+1] = user_col
        sample_df['item_id'][split_index[0]:split_index[-1]+1] = zero_samples
        sample_df['rating'][split_index[0]:split_index[-1]+1] = zero_col
        sample_df['context_feature_id'][split_index[0]:split_index[-1]+1] = context_features


    return pd.DataFrame(sample_df)

In [56]:
def sample_zeros(user_df, k, all_items):
    sample = []
    
    while len(sample) < k*len(user_df[:,1]):
        raw_samp = np.random.randint(0, len(all_items))
        if raw_samp not in user_df[:,1] and raw_samp not in sample:
            sample.append(raw_samp)

    return sample

In [322]:
train_df = pd.read_csv('training.csv')
train_df['rating'] = 1
items_id_df = pd.read_csv('item_feature.csv')

In [214]:
sampling = train_df.to_numpy()
all_users = pd.unique(sampling[:,0])
all_items = pd.unique(sampling[:,1])

In [215]:
negative_samples = add_samples(sampling, 1, all_users, all_items)

In [216]:
context_feature_id_dict = {}
for user in train_df['user_id'].unique():
    context_feature_id_dict[user] = train_df[train_df['user_id'] == user]['context_feature_id'].sample(1).values[0]

In [217]:
negative_samples['context_feature_id'] = negative_samples['user_id'].map(context_feature_id_dict)
train_df_new = pd.concat([negative_samples, train_df])

In [242]:
training_df = train_df_new.merge(items_id_df, left_on='item_id', right_on='item_id')
training_df = training_df.astype('int')
training_df

,user_id,item_id,rating,context_feature_id,item_feature_id
0,0,22331,0,3,124
1,3323,22331,0,2,124
2,6421,22331,0,0,124
3,9189,22331,0,2,124
4,19281,22331,0,3,124
...,...,...,...,...,...
1940485,198093,38949,1,1,142
1940486,198439,39733,1,2,138
1940487,198921,39831,1,2,94
1940488,198921,37989,1,2,94


In [285]:
training_df

,user_id,item_id,rating,context_feature_id,item_feature_id
0,0,22331,0,3,124
1,3323,22331,0,2,124
2,6421,22331,0,0,124
3,9189,22331,0,2,124
4,19281,22331,0,3,124
...,...,...,...,...,...
1940485,198093,38949,1,1,142
1940486,198439,39733,1,2,138
1940487,198921,39831,1,2,94
1940488,198921,37989,1,2,94


### Matrix Factorization Model
---

In [57]:
class MF_bias(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100, seed=None):
        super(MF_bias, self).__init__()
        if seed is None:
            torch.manual_seed(np.random.randint(1,100000))
        else:
            torch.manual_seed(seed)
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.user_bias = nn.Embedding(num_users, 1)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.item_bias = nn.Embedding(num_items, 1)
        
        self.user_emb.weight.data.uniform_(0,0.05)
        self.item_emb.weight.data.uniform_(0,0.05)
        self.user_bias.weight.data.uniform_(-0.01,0.01)
        self.item_bias.weight.data.uniform_(-0.01,0.01)

    def forward(self, u, v):
        U = self.user_emb(u)
        V = self.item_emb(v)
        b_u = self.user_bias(u).squeeze()
        b_v = self.item_bias(v).squeeze()
        return ((U*V).sum(1) + b_u + b_v)

In [58]:
def train_epocs(model, epochs=10, lr=0.01, wd=0.0):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    for i in range(epochs):
        model.train()
        users = torch.LongTensor(train_df.user_id.values)  
        items = torch.LongTensor(train_df.item_id.values) 
        ratings = torch.FloatTensor(train_df.rating.values)  
    
        y_hat = model(users, items)
        loss = F.binary_cross_entropy_with_logits(y_hat, ratings)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        testloss, acc = valid_loss(model)
        print("train loss %.3f valid loss %.3f accuracy %.3f" % (loss.item(), testloss, acc)) 

In [59]:
def train_epocs_full(model, epochs=10, lr=0.01, wd=0.0):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    for i in range(epochs):
        model.train()
        users = torch.LongTensor(training_df.user_id.values)  
        items = torch.LongTensor(training_df.item_id.values) 
        ratings = torch.FloatTensor(training_df.rating.values)  
    
        y_hat = model(users, items)
        loss = F.binary_cross_entropy_with_logits(y_hat, ratings)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Trained!')

In [60]:
def valid_loss(model):
    model.eval()
    users = torch.LongTensor(valid_df.user_id.values)  
    items = torch.LongTensor(valid_df.item_id.values) 
    ratings = torch.FloatTensor(valid_df.rating.values)  
    y_hat = model(users, items)
    loss = F.binary_cross_entropy_with_logits(y_hat, ratings)
    preds = (torch.sigmoid(y_hat) > 0.5).float()
    return loss.item(), balanced_accuracy_score(ratings, preds)

In [61]:
training_df = pd.read_feather('training_sampled')
training_df

,user_id,item_id,rating,context_feature_id,item_feature_id
0,0,22331,0,3,124
1,3323,22331,0,2,124
2,6421,22331,0,0,124
3,9189,22331,0,2,124
4,19281,22331,0,3,124
...,...,...,...,...,...
1940485,198093,38949,1,1,142
1940486,198439,39733,1,2,138
1940487,198921,39831,1,2,94
1940488,198921,37989,1,2,94


In [62]:
train_df, valid_df = train_test_split(training_df)

In [60]:
num_users = training_df['user_id'].max()
num_items = training_df['item_id'].unique().max()
model = MF_bias(num_users+1, num_items+1, emb_size=25)

In [61]:
train_epocs(model, 80, 0.1, 1e-5)

train loss 0.693 valid loss 0.745 accuracy 0.184
train loss 0.711 valid loss 0.650 accuracy 0.866
train loss 0.627 valid loss 0.602 accuracy 0.869
train loss 0.594 valid loss 0.577 accuracy 0.872
train loss 0.576 valid loss 0.546 accuracy 0.873
train loss 0.546 valid loss 0.525 accuracy 0.872
train loss 0.519 valid loss 0.518 accuracy 0.867
train loss 0.505 valid loss 0.517 accuracy 0.863
train loss 0.498 valid loss 0.509 accuracy 0.862
train loss 0.489 valid loss 0.494 accuracy 0.867
train loss 0.475 valid loss 0.479 accuracy 0.869
train loss 0.463 valid loss 0.468 accuracy 0.870
train loss 0.455 valid loss 0.462 accuracy 0.872
train loss 0.452 valid loss 0.459 accuracy 0.873
train loss 0.450 valid loss 0.458 accuracy 0.872
train loss 0.448 valid loss 0.460 accuracy 0.871
train loss 0.447 valid loss 0.463 accuracy 0.870
train loss 0.447 valid loss 0.467 accuracy 0.869
train loss 0.449 valid loss 0.468 accuracy 0.869
train loss 0.450 valid loss 0.466 accuracy 0.869
train loss 0.450 val

In [62]:
train_epocs(model, 150, lr=0.005, wd=1e-6)

train loss 0.420 valid loss 0.430 accuracy 0.871
train loss 0.419 valid loss 0.429 accuracy 0.871
train loss 0.417 valid loss 0.428 accuracy 0.871
train loss 0.416 valid loss 0.427 accuracy 0.871
train loss 0.415 valid loss 0.426 accuracy 0.871
train loss 0.413 valid loss 0.425 accuracy 0.871
train loss 0.412 valid loss 0.424 accuracy 0.871
train loss 0.411 valid loss 0.423 accuracy 0.871
train loss 0.409 valid loss 0.422 accuracy 0.871
train loss 0.408 valid loss 0.421 accuracy 0.871
train loss 0.407 valid loss 0.420 accuracy 0.871
train loss 0.405 valid loss 0.420 accuracy 0.871
train loss 0.404 valid loss 0.418 accuracy 0.871
train loss 0.402 valid loss 0.417 accuracy 0.871
train loss 0.401 valid loss 0.416 accuracy 0.871
train loss 0.400 valid loss 0.415 accuracy 0.871
train loss 0.398 valid loss 0.414 accuracy 0.871
train loss 0.396 valid loss 0.413 accuracy 0.871
train loss 0.395 valid loss 0.412 accuracy 0.871
train loss 0.393 valid loss 0.411 accuracy 0.871
train loss 0.391 val

In [63]:
train_epocs(model, 10, lr=0.005, wd=1e-6)

train loss 0.166 valid loss 0.305 accuracy 0.901
train loss 0.166 valid loss 0.304 accuracy 0.901
train loss 0.165 valid loss 0.304 accuracy 0.901
train loss 0.165 valid loss 0.303 accuracy 0.901
train loss 0.164 valid loss 0.303 accuracy 0.901
train loss 0.164 valid loss 0.302 accuracy 0.901
train loss 0.163 valid loss 0.302 accuracy 0.901
train loss 0.163 valid loss 0.302 accuracy 0.902
train loss 0.162 valid loss 0.301 accuracy 0.902
train loss 0.162 valid loss 0.301 accuracy 0.902


In [64]:
train_epocs(model, 10, lr=0.005, wd=1e-6)

train loss 0.162 valid loss 0.300 accuracy 0.902
train loss 0.161 valid loss 0.300 accuracy 0.902
train loss 0.161 valid loss 0.300 accuracy 0.902
train loss 0.160 valid loss 0.299 accuracy 0.902
train loss 0.160 valid loss 0.299 accuracy 0.902
train loss 0.160 valid loss 0.298 accuracy 0.902
train loss 0.159 valid loss 0.298 accuracy 0.902
train loss 0.159 valid loss 0.298 accuracy 0.902
train loss 0.158 valid loss 0.297 accuracy 0.902
train loss 0.158 valid loss 0.297 accuracy 0.903


In [65]:
train_epocs(model, 10, lr=0.005, wd=1e-6)

train loss 0.158 valid loss 0.297 accuracy 0.903
train loss 0.157 valid loss 0.296 accuracy 0.903
train loss 0.157 valid loss 0.296 accuracy 0.903
train loss 0.157 valid loss 0.296 accuracy 0.903
train loss 0.156 valid loss 0.295 accuracy 0.903
train loss 0.156 valid loss 0.295 accuracy 0.903
train loss 0.156 valid loss 0.294 accuracy 0.903
train loss 0.155 valid loss 0.294 accuracy 0.903
train loss 0.155 valid loss 0.294 accuracy 0.903
train loss 0.155 valid loss 0.293 accuracy 0.904


In [66]:
train_epocs(model, 10, lr=0.005, wd=1e-6)

train loss 0.154 valid loss 0.293 accuracy 0.904
train loss 0.154 valid loss 0.293 accuracy 0.904
train loss 0.154 valid loss 0.292 accuracy 0.904
train loss 0.154 valid loss 0.292 accuracy 0.904
train loss 0.153 valid loss 0.292 accuracy 0.904
train loss 0.153 valid loss 0.291 accuracy 0.904
train loss 0.153 valid loss 0.291 accuracy 0.904
train loss 0.152 valid loss 0.291 accuracy 0.904
train loss 0.152 valid loss 0.291 accuracy 0.904
train loss 0.152 valid loss 0.290 accuracy 0.904


In [67]:
train_epocs(model, 10, lr=0.005, wd=1e-6)

train loss 0.152 valid loss 0.290 accuracy 0.905
train loss 0.151 valid loss 0.290 accuracy 0.905
train loss 0.151 valid loss 0.289 accuracy 0.905
train loss 0.151 valid loss 0.289 accuracy 0.905
train loss 0.151 valid loss 0.289 accuracy 0.905
train loss 0.150 valid loss 0.288 accuracy 0.905
train loss 0.150 valid loss 0.288 accuracy 0.905
train loss 0.150 valid loss 0.288 accuracy 0.905
train loss 0.150 valid loss 0.288 accuracy 0.905
train loss 0.149 valid loss 0.287 accuracy 0.905


In [68]:
train_epocs(model, 10, lr=0.005, wd=1e-6)

train loss 0.149 valid loss 0.287 accuracy 0.905
train loss 0.149 valid loss 0.287 accuracy 0.905
train loss 0.149 valid loss 0.286 accuracy 0.906
train loss 0.148 valid loss 0.286 accuracy 0.906
train loss 0.148 valid loss 0.286 accuracy 0.906
train loss 0.148 valid loss 0.286 accuracy 0.906
train loss 0.148 valid loss 0.285 accuracy 0.906
train loss 0.148 valid loss 0.285 accuracy 0.906
train loss 0.147 valid loss 0.285 accuracy 0.906
train loss 0.147 valid loss 0.285 accuracy 0.906


In [69]:
train_epocs(model, 10, lr=0.005, wd=1e-6)

train loss 0.147 valid loss 0.284 accuracy 0.906
train loss 0.147 valid loss 0.284 accuracy 0.906
train loss 0.147 valid loss 0.284 accuracy 0.906
train loss 0.146 valid loss 0.284 accuracy 0.906
train loss 0.146 valid loss 0.283 accuracy 0.906
train loss 0.146 valid loss 0.283 accuracy 0.907
train loss 0.146 valid loss 0.283 accuracy 0.907
train loss 0.146 valid loss 0.283 accuracy 0.907
train loss 0.146 valid loss 0.282 accuracy 0.907
train loss 0.145 valid loss 0.282 accuracy 0.907


In [70]:
train_epocs(model, 10, lr=0.005, wd=1e-6)

train loss 0.145 valid loss 0.282 accuracy 0.907
train loss 0.145 valid loss 0.282 accuracy 0.907
train loss 0.145 valid loss 0.282 accuracy 0.907
train loss 0.145 valid loss 0.281 accuracy 0.907
train loss 0.145 valid loss 0.281 accuracy 0.907
train loss 0.144 valid loss 0.281 accuracy 0.907
train loss 0.144 valid loss 0.281 accuracy 0.907
train loss 0.144 valid loss 0.281 accuracy 0.907
train loss 0.144 valid loss 0.280 accuracy 0.907
train loss 0.144 valid loss 0.280 accuracy 0.907


In [71]:
train_epocs(model, 10, lr=0.005, wd=1e-6)

train loss 0.144 valid loss 0.280 accuracy 0.907
train loss 0.144 valid loss 0.280 accuracy 0.908
train loss 0.143 valid loss 0.280 accuracy 0.908
train loss 0.143 valid loss 0.280 accuracy 0.908
train loss 0.143 valid loss 0.279 accuracy 0.908
train loss 0.143 valid loss 0.279 accuracy 0.908
train loss 0.143 valid loss 0.279 accuracy 0.908
train loss 0.143 valid loss 0.279 accuracy 0.908
train loss 0.143 valid loss 0.279 accuracy 0.908
train loss 0.143 valid loss 0.279 accuracy 0.908


In [72]:
train_epocs(model, 10, lr=0.005, wd=1e-6)

train loss 0.142 valid loss 0.279 accuracy 0.908
train loss 0.142 valid loss 0.278 accuracy 0.908
train loss 0.142 valid loss 0.278 accuracy 0.908
train loss 0.142 valid loss 0.278 accuracy 0.908
train loss 0.142 valid loss 0.278 accuracy 0.908
train loss 0.142 valid loss 0.278 accuracy 0.908
train loss 0.142 valid loss 0.278 accuracy 0.908
train loss 0.142 valid loss 0.278 accuracy 0.908
train loss 0.142 valid loss 0.277 accuracy 0.908
train loss 0.142 valid loss 0.277 accuracy 0.908


In [73]:
train_epocs(model, 10, lr=0.005, wd=1e-6)

train loss 0.141 valid loss 0.277 accuracy 0.908
train loss 0.141 valid loss 0.277 accuracy 0.908
train loss 0.141 valid loss 0.277 accuracy 0.908
train loss 0.141 valid loss 0.277 accuracy 0.908
train loss 0.141 valid loss 0.277 accuracy 0.908
train loss 0.141 valid loss 0.277 accuracy 0.909
train loss 0.141 valid loss 0.277 accuracy 0.909
train loss 0.141 valid loss 0.277 accuracy 0.909
train loss 0.141 valid loss 0.276 accuracy 0.909
train loss 0.141 valid loss 0.276 accuracy 0.909


In [74]:
train_epocs(model, 10, lr=0.005, wd=1e-6)

train loss 0.141 valid loss 0.276 accuracy 0.909
train loss 0.141 valid loss 0.276 accuracy 0.909
train loss 0.141 valid loss 0.276 accuracy 0.909
train loss 0.141 valid loss 0.276 accuracy 0.909
train loss 0.141 valid loss 0.276 accuracy 0.909
train loss 0.140 valid loss 0.276 accuracy 0.909
train loss 0.140 valid loss 0.276 accuracy 0.909
train loss 0.140 valid loss 0.276 accuracy 0.909
train loss 0.140 valid loss 0.276 accuracy 0.909
train loss 0.140 valid loss 0.276 accuracy 0.909


In [75]:
train_epocs(model, 10, lr=0.005, wd=1e-6)

train loss 0.140 valid loss 0.276 accuracy 0.909
train loss 0.140 valid loss 0.276 accuracy 0.909
train loss 0.140 valid loss 0.275 accuracy 0.909
train loss 0.140 valid loss 0.275 accuracy 0.909
train loss 0.140 valid loss 0.275 accuracy 0.909
train loss 0.140 valid loss 0.275 accuracy 0.909
train loss 0.140 valid loss 0.275 accuracy 0.909
train loss 0.140 valid loss 0.275 accuracy 0.909
train loss 0.140 valid loss 0.275 accuracy 0.909
train loss 0.140 valid loss 0.275 accuracy 0.909


In [58]:
train_epocs(model, 10, lr=0.005, wd=1e-6)

train loss 0.169 valid loss 0.299 accuracy 0.901
train loss 0.168 valid loss 0.298 accuracy 0.901
train loss 0.168 valid loss 0.298 accuracy 0.901
train loss 0.167 valid loss 0.297 accuracy 0.901
train loss 0.167 valid loss 0.297 accuracy 0.902
train loss 0.166 valid loss 0.297 accuracy 0.902
train loss 0.165 valid loss 0.296 accuracy 0.902
train loss 0.165 valid loss 0.296 accuracy 0.902
train loss 0.164 valid loss 0.295 accuracy 0.902
train loss 0.164 valid loss 0.295 accuracy 0.902


In [59]:
train_epocs(model, 10, lr=0.005, wd=1e-6)

train loss 0.163 valid loss 0.295 accuracy 0.903
train loss 0.163 valid loss 0.294 accuracy 0.903
train loss 0.162 valid loss 0.294 accuracy 0.903
train loss 0.162 valid loss 0.294 accuracy 0.903
train loss 0.161 valid loss 0.293 accuracy 0.903
train loss 0.161 valid loss 0.293 accuracy 0.903
train loss 0.160 valid loss 0.292 accuracy 0.903
train loss 0.160 valid loss 0.292 accuracy 0.903
train loss 0.159 valid loss 0.292 accuracy 0.904
train loss 0.159 valid loss 0.291 accuracy 0.904


In [19]:
# ============================ #
# HYPER PARAMETER EXPERIMENTATION
# ============================ #
model2 = MF_bias(num_users, num_items, emb_size=200)
train_epocs(model2, epochs=10, lr=0.1)

train loss 0.695 valid loss 0.684 accuracy 0.607
train loss 0.605 valid loss 0.413 accuracy 0.850
train loss 0.293 valid loss 0.442 accuracy 0.847
train loss 0.210 valid loss 0.478 accuracy 0.845
train loss 0.131 valid loss 0.496 accuracy 0.845
train loss 0.072 valid loss 0.514 accuracy 0.849
train loss 0.037 valid loss 0.544 accuracy 0.851
train loss 0.019 valid loss 0.586 accuracy 0.851
train loss 0.010 valid loss 0.639 accuracy 0.852
train loss 0.005 valid loss 0.696 accuracy 0.852


In [20]:
model3 = MF_bias(num_users, num_items, emb_size=200)
train_epocs(model3, epochs=10, lr=0.01)

train loss 0.695 valid loss 0.681 accuracy 0.500
train loss 0.671 valid loss 0.665 accuracy 0.591
train loss 0.646 valid loss 0.643 accuracy 0.843
train loss 0.617 valid loss 0.614 accuracy 0.857
train loss 0.582 valid loss 0.579 accuracy 0.857
train loss 0.541 valid loss 0.541 accuracy 0.856
train loss 0.498 valid loss 0.501 accuracy 0.857
train loss 0.454 valid loss 0.462 accuracy 0.858
train loss 0.411 valid loss 0.429 accuracy 0.858
train loss 0.374 valid loss 0.402 accuracy 0.858


In [21]:
train_epocs(model3, epochs=5, lr=0.005)

train loss 0.342 valid loss 0.390 accuracy 0.858
train loss 0.327 valid loss 0.380 accuracy 0.858
train loss 0.313 valid loss 0.372 accuracy 0.858
train loss 0.300 valid loss 0.366 accuracy 0.858
train loss 0.289 valid loss 0.360 accuracy 0.857


In [22]:
train_epocs(model3, epochs=5, lr=0.005)

train loss 0.279 valid loss 0.356 accuracy 0.857
train loss 0.269 valid loss 0.353 accuracy 0.857
train loss 0.259 valid loss 0.351 accuracy 0.857
train loss 0.250 valid loss 0.349 accuracy 0.857
train loss 0.242 valid loss 0.348 accuracy 0.857


In [23]:
train_epocs(model3, epochs=5, lr=0.005)

train loss 0.234 valid loss 0.347 accuracy 0.858
train loss 0.225 valid loss 0.347 accuracy 0.858
train loss 0.217 valid loss 0.346 accuracy 0.858
train loss 0.209 valid loss 0.346 accuracy 0.858
train loss 0.202 valid loss 0.347 accuracy 0.858


In [24]:
model4 = MF_bias(num_users, num_items, emb_size=500)
train_epocs(model4, epochs=15, lr=0.01)

train loss 0.705 valid loss 0.671 accuracy 0.500
train loss 0.654 valid loss 0.641 accuracy 0.503
train loss 0.610 valid loss 0.603 accuracy 0.855
train loss 0.561 valid loss 0.556 accuracy 0.859
train loss 0.505 valid loss 0.504 accuracy 0.857
train loss 0.446 valid loss 0.455 accuracy 0.857
train loss 0.391 valid loss 0.415 accuracy 0.858
train loss 0.343 valid loss 0.387 accuracy 0.858
train loss 0.306 valid loss 0.372 accuracy 0.858
train loss 0.279 valid loss 0.368 accuracy 0.858
train loss 0.258 valid loss 0.370 accuracy 0.858
train loss 0.241 valid loss 0.374 accuracy 0.858
train loss 0.226 valid loss 0.380 accuracy 0.858
train loss 0.210 valid loss 0.385 accuracy 0.858
train loss 0.194 valid loss 0.390 accuracy 0.858


In [76]:
#train on full data
model = MF_bias(num_users+1, num_items+1, emb_size=25)
train_epocs_full(model, 80, 0.1, 1e-5)

Trained!


In [77]:
train_epocs_full(model, 260, lr=0.005, wd=1e-6)

Trained!


In [63]:
num_users = training_df['user_id'].max()
num_items = training_df['item_id'].unique().max()
model = MF_bias(num_users+1, num_items+1, emb_size=75)

In [64]:
train_epocs_full(model, 100, 0.1, 1e-4)

Trained!


In [71]:
train_epocs_full(model, 150, 0.005, 1e-6)

Trained!


### Round 3 Experimentation


In [29]:
class MF_features_bias(nn.Module):
    def __init__(self, num_users, num_items, num_context, num_item_features, emb_size=100, feat_emb_size=50, seed=None):
        super(MF_features_bias, self).__init__()
        if seed is None:
            torch.manual_seed(np.random.randint(1,100000))
        else:
            torch.manual_seed(seed)
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.user_bias = nn.Embedding(num_users, 1)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.item_bias = nn.Embedding(num_items, 1)
        self.context_emb = nn.Embedding(num_context, feat_emb_size)
        self.context_bias = nn.Embedding(num_context, 1)
        self.item_feature_emb = nn.Embedding(num_item_features, feat_emb_size)
        self.item_feature_bias = nn.Embedding(num_item_features, 1)
        
        self.user_emb.weight.data.uniform_(0,0.05)
        self.item_emb.weight.data.uniform_(0,0.05)
        self.context_emb.weight.data.uniform_(0,0.05)
        self.item_feature_emb.weight.data.uniform_(0,0.05)
        self.user_bias.weight.data.uniform_(-0.01,0.01)
        self.item_bias.weight.data.uniform_(-0.01,0.01)
        self.context_bias.weight.data.uniform_(-0.01,0.01)
        self.item_feature_bias.weight.data.uniform_(-0.01,0.01)

    def forward(self, u, v, c, i):
        U = self.user_emb(u)
        V = self.item_emb(v)
        C = self.context_emb(c)
        I = self.item_feature_emb(i)
        b_u = self.user_bias(u).squeeze()
        b_v = self.item_bias(v).squeeze()
        b_c = self.user_bias(u).squeeze()
        b_i = self.item_bias(v).squeeze()
        return ((U*V).sum(1) + ((C*I).sum(1)) + b_u + b_v + b_c + b_i)

In [20]:
def train_epocs(model, epochs=10, lr=0.01, wd=0.0):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    for i in range(epochs):
        model.train()
        users = torch.LongTensor(train_df.user_id.values)  
        items = torch.LongTensor(train_df.item_id.values) 
        context_features = torch.LongTensor(train_df.context_feature_id.values)  
        item_features = torch.LongTensor(train_df.item_feature_id.values) 
        ratings = torch.FloatTensor(train_df.rating.values)  
    
        y_hat = model(users, items, context_features, item_features)
        loss = F.binary_cross_entropy_with_logits(y_hat, ratings)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        testloss, acc = valid_loss(model)
        print("train loss %.3f valid loss %.3f accuracy %.3f" % (loss.item(), testloss, acc)) 

In [32]:
def valid_loss(model):
    model.eval()
    users = torch.LongTensor(valid_df.user_id.values)  
    items = torch.LongTensor(valid_df.item_id.values)
    context_features = torch.LongTensor(valid_df.context_feature_id.values)  
    item_features = torch.LongTensor(valid_df.item_feature_id.values) 
    ratings = torch.FloatTensor(valid_df.rating.values)  
    y_hat = model(users, items, context_features, item_features)
    loss = F.binary_cross_entropy_with_logits(y_hat, ratings)
    preds = (torch.sigmoid(y_hat) > 0.5).float()
    return loss.item(), balanced_accuracy_score(ratings, preds)

In [27]:
valid_df

,user_id,item_id,rating,context_feature_id,item_feature_id
0,63411,22331,0,2,124
1,95998,22331,0,0,124
2,118432,22331,0,2,124
3,127335,22331,1,1,124
4,0,14064,0,3,75
...,...,...,...,...,...
194074,189501,38356,1,2,63
194075,194022,38179,1,2,33
194076,194817,38088,1,0,148
194077,197009,39877,1,3,173


In [30]:
num_users = training_df['user_id'].max()
num_items = training_df['item_id'].unique().max()
num_context = training_df['context_feature_id'].max()
num_item_features = training_df['item_feature_id'].max()
model = MF_features_bias(num_users+1, num_items+1, num_context+1, num_item_features+1, emb_size=25, feat_emb_size=25)

In [33]:
train_epocs(model, 80, 0.1, 1e-5)

train loss 0.677 valid loss 0.552 accuracy 0.783
train loss 0.545 valid loss 0.507 accuracy 0.810
train loss 0.505 valid loss 0.470 accuracy 0.847
train loss 0.456 valid loss 0.442 accuracy 0.851
train loss 0.415 valid loss 0.425 accuracy 0.853
train loss 0.390 valid loss 0.413 accuracy 0.856
train loss 0.376 valid loss 0.397 accuracy 0.861
train loss 0.361 valid loss 0.380 accuracy 0.866
train loss 0.348 valid loss 0.367 accuracy 0.868
train loss 0.339 valid loss 0.357 accuracy 0.869
train loss 0.332 valid loss 0.352 accuracy 0.869
train loss 0.329 valid loss 0.349 accuracy 0.870
train loss 0.327 valid loss 0.347 accuracy 0.870
train loss 0.325 valid loss 0.346 accuracy 0.869
train loss 0.323 valid loss 0.347 accuracy 0.868
train loss 0.322 valid loss 0.348 accuracy 0.867
train loss 0.321 valid loss 0.349 accuracy 0.867
train loss 0.321 valid loss 0.350 accuracy 0.866
train loss 0.321 valid loss 0.350 accuracy 0.865
train loss 0.321 valid loss 0.349 accuracy 0.866
train loss 0.321 val

In [34]:
train_epocs(model, 150, lr=0.005, wd=1e-6)

train loss 0.307 valid loss 0.331 accuracy 0.868
train loss 0.306 valid loss 0.330 accuracy 0.868
train loss 0.305 valid loss 0.330 accuracy 0.868
train loss 0.304 valid loss 0.329 accuracy 0.868
train loss 0.303 valid loss 0.329 accuracy 0.868
train loss 0.302 valid loss 0.329 accuracy 0.868
train loss 0.301 valid loss 0.328 accuracy 0.869
train loss 0.300 valid loss 0.328 accuracy 0.869
train loss 0.299 valid loss 0.327 accuracy 0.869
train loss 0.298 valid loss 0.327 accuracy 0.869
train loss 0.297 valid loss 0.327 accuracy 0.869
train loss 0.296 valid loss 0.326 accuracy 0.869
train loss 0.295 valid loss 0.326 accuracy 0.869
train loss 0.294 valid loss 0.326 accuracy 0.869
train loss 0.293 valid loss 0.325 accuracy 0.869
train loss 0.292 valid loss 0.325 accuracy 0.869
train loss 0.291 valid loss 0.325 accuracy 0.869
train loss 0.290 valid loss 0.324 accuracy 0.870
train loss 0.289 valid loss 0.324 accuracy 0.870
train loss 0.288 valid loss 0.324 accuracy 0.870
train loss 0.287 val

#### Averaging Models

In [38]:
pred_1 = pd.read_csv('submission.csv')
pred_2 = pd.read_csv('submission_2.csv')

0.6427675692269349

## Predicting for Observed Users

In [66]:
#Testing
test = pd.read_csv('test_kaggle.csv')
test_ids = test['id']
items_id_df = pd.read_csv('item_feature.csv')

In [44]:
test = test.merge(items_id_df, left_on='item_id', right_on='item_id')

In [45]:
users = torch.LongTensor(test.user_id.values)  
items = torch.LongTensor(test.item_id.values)
context_features = torch.LongTensor(test.context_feature_id.values)  
item_features = torch.LongTensor(test.item_feature_id.values) 

y_hat_final = torch.sigmoid(model(users, items, context_features, item_features))

In [72]:
users = torch.LongTensor(test.user_id.values)  
items = torch.LongTensor(test.item_id.values)

y_hat_final = torch.sigmoid(model(users, items))

In [73]:
kaggle_df = pd.DataFrame(list(y_hat_final.detach().numpy()), index=test_ids).rename({0: 'rating'}, axis='columns')
kaggle_df

,rating
id,
0,0.604041
1,0.414341
2,0.847360
3,0.275295
4,0.230527
...,...
381380,0.781686
381381,0.932825
381382,0.892413


In [74]:
kaggle_df['rating'].mean()

0.5824710726737976

In [75]:
kaggle_df.to_csv('submission_6.csv')

### NN Model 2
***

#### Data Pre-Processing

In [213]:
train_df = pd.read_csv('training.csv')
train_df['rating'] = 1
items_id_df = pd.read_csv('item_feature.csv')

In [214]:
sampling = train_df.to_numpy()
all_users = pd.unique(sampling[:,0])
all_items = pd.unique(sampling[:,1])

In [215]:
negative_samples = add_samples(sampling, 1, all_users, all_items)

In [216]:
context_feature_id_dict = {}
for user in train_df['user_id'].unique():
    context_feature_id_dict[user] = train_df[train_df['user_id'] == user]['context_feature_id'].sample(1).values[0]

In [217]:
negative_samples['context_feature_id'] = negative_samples['user_id'].map(context_feature_id_dict)
train_df_new = pd.concat([negative_samples, train_df])

In [242]:
training_df = train_df_new.merge(items_id_df, left_on='item_id', right_on='item_id')
training_df = training_df.astype('int')
training_df

,user_id,item_id,rating,context_feature_id,item_feature_id
0,0,22331,0,3,124
1,3323,22331,0,2,124
2,6421,22331,0,0,124
3,9189,22331,0,2,124
4,19281,22331,0,3,124
...,...,...,...,...,...
1940485,198093,38949,1,1,142
1940486,198439,39733,1,2,138
1940487,198921,39831,1,2,94
1940488,198921,37989,1,2,94


In [221]:
training_df.to_feather('training_sampled')

#### Modeling

In [231]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [232]:
def train_epocs(model, epochs=10, lr=0.01, wd=0.0):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    for i in range(epochs):
        model.train()
        users = torch.LongTensor(train_df_NN.user_id.values)  
        items = torch.LongTensor(train_df_NN.item_id.values) 
        context_feature_id = torch.LongTensor(train_df_NN.context_feature_id.values)
        item_feature_id = torch.LongTensor(train_df_NN.item_feature_id.values)
        ratings = torch.FloatTensor(train_df_NN.rating.values)  
    
        y_hat = model(users, items, item_feature_id, context_feature_id)
        loss = F.binary_cross_entropy_with_logits(y_hat, ratings)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        testloss, acc = valid_loss(model)
        print("train loss %.3f valid loss %.3f accuracy %.3f" % (loss.item(), testloss, acc)) 

In [233]:
def valid_loss(model):
    model.eval()
    users = torch.LongTensor(test_df_NN.user_id.values)  
    items = torch.LongTensor(test_df_NN.item_id.values) 
    context_feature_id = torch.LongTensor(test_df_NN.context_feature_id.values)
    item_feature_id = torch.LongTensor(test_df_NN.item_feature_id.values)
    ratings = torch.FloatTensor(test_df_NN.rating.values)

    y_hat = model(users, items, item_feature_id, context_feature_id)
    loss = F.binary_cross_entropy_with_logits(y_hat, ratings)
    preds = (y_hat > 0.5).float()
    return loss.item(), balanced_accuracy_score(ratings, preds)

In [234]:
class NN(nn.Module):
    def __init__(self, num_users, num_items, num_item_features, num_context_features, emb_size=75, M=2):
        super(NN, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.item_feature_id = nn.Embedding(num_item_features, emb_size)
        self.context_feature_id = nn.Embedding(num_context_features, emb_size)
        self.hidden_layer = M
        # initlializing weights
        self.user_emb.weight.data.uniform_(0,0.05)
        self.item_emb.weight.data.uniform_(0,0.05)
        self.item_feature_id.weight.data.uniform_(0,0.05)
        self.context_feature_id.weight.data.uniform_(0,0.05)
        #layers
        self.linear1 = nn.Linear(emb_size*4, M)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(M, 1)
        self.sig = nn.Sigmoid()
        
    def forward(self, u, v, i, c):
        u = self.user_emb(u)
        v = self.item_emb(v)
        i = self.item_feature_id(i)
        c = self.context_feature_id(c)
        concat = torch.concat([u, v, i, c], axis=1)
        pred = self.linear1(concat)
        pred = self.relu(pred)
        pred = self.linear2(pred)
        return self.sig(pred.squeeze())

In [243]:
train_df_NN, test_df_NN = train_test_split(training_df)
train_df_NN = train_df_NN.drop(columns=['index'])
test_df_NN = test_df_NN.drop(columns=['index'])

In [245]:
X_train = train_df_NN.drop(columns='rating').values
y_train = train_df_NN['rating']
X_test = test_df_NN.drop(columns='rating').values
y_test = test_df_NN['rating']

In [246]:
num_users = training_df['user_id'].max()
num_items = training_df['item_id'].max()
num_item_features = training_df['item_feature_id'].max()
num_context_features = training_df['context_feature_id'].max()
model = NN(num_users+1, num_items+1, num_item_features+1, num_context_features+1, emb_size=25, M=3)

In [247]:
train_epocs(model, lr=0.08)

train loss 0.717 valid loss 0.712 accuracy 0.500
train loss 0.713 valid loss 0.699 accuracy 0.500
train loss 0.698 valid loss 0.681 accuracy 0.500
train loss 0.679 valid loss 0.669 accuracy 0.500
train loss 0.665 valid loss 0.659 accuracy 0.500
train loss 0.653 valid loss 0.642 accuracy 0.500
train loss 0.637 valid loss 0.636 accuracy 0.500
train loss 0.634 valid loss 0.633 accuracy 0.500
train loss 0.631 valid loss 0.631 accuracy 0.500
train loss 0.629 valid loss 0.629 accuracy 0.500


In [181]:
train_epocs(model, lr=0.005)

train loss 0.571 valid loss 0.578 accuracy 0.859
train loss 0.569 valid loss 0.576 accuracy 0.859
train loss 0.568 valid loss 0.575 accuracy 0.859
train loss 0.567 valid loss 0.574 accuracy 0.860
train loss 0.566 valid loss 0.573 accuracy 0.860
train loss 0.565 valid loss 0.572 accuracy 0.860
train loss 0.565 valid loss 0.572 accuracy 0.860
train loss 0.564 valid loss 0.571 accuracy 0.860
train loss 0.564 valid loss 0.571 accuracy 0.860
train loss 0.563 valid loss 0.570 accuracy 0.860


In [248]:
model = NN(num_users+1, num_items+1, num_item_features+1, num_context_features+1, emb_size=25, M=3)

In [249]:
train_epocs(model, lr=0.1, wd=1e-5)

train loss 0.719 valid loss 0.716 accuracy 0.500
train loss 0.716 valid loss 0.711 accuracy 0.500
train loss 0.711 valid loss 0.703 accuracy 0.500
train loss 0.703 valid loss 0.688 accuracy 0.500
train loss 0.688 valid loss 0.677 accuracy 0.500
train loss 0.677 valid loss 0.674 accuracy 0.500
train loss 0.674 valid loss 0.667 accuracy 0.500
train loss 0.667 valid loss 0.657 accuracy 0.500
train loss 0.657 valid loss 0.647 accuracy 0.542
train loss 0.647 valid loss 0.632 accuracy 0.645


In [250]:
train_epocs(model, lr=0.05, wd=1e-5)

train loss 0.632 valid loss 0.615 accuracy 0.725
train loss 0.615 valid loss 0.602 accuracy 0.749
train loss 0.602 valid loss 0.592 accuracy 0.785
train loss 0.592 valid loss 0.586 accuracy 0.801
train loss 0.586 valid loss 0.581 accuracy 0.817
train loss 0.580 valid loss 0.576 accuracy 0.829
train loss 0.575 valid loss 0.572 accuracy 0.840
train loss 0.571 valid loss 0.569 accuracy 0.845
train loss 0.568 valid loss 0.568 accuracy 0.847
train loss 0.567 valid loss 0.567 accuracy 0.851


In [251]:
train_epocs(model, lr=0.01, wd=1e-5)

train loss 0.565 valid loss 0.566 accuracy 0.853
train loss 0.565 valid loss 0.567 accuracy 0.848
train loss 0.564 valid loss 0.566 accuracy 0.851
train loss 0.564 valid loss 0.566 accuracy 0.856
train loss 0.563 valid loss 0.565 accuracy 0.859
train loss 0.562 valid loss 0.565 accuracy 0.860
train loss 0.561 valid loss 0.565 accuracy 0.860
train loss 0.560 valid loss 0.564 accuracy 0.861
train loss 0.560 valid loss 0.564 accuracy 0.863
train loss 0.559 valid loss 0.564 accuracy 0.864


In [252]:
train_epocs(model, lr=0.01, wd=1e-5)

train loss 0.559 valid loss 0.566 accuracy 0.850
train loss 0.561 valid loss 0.564 accuracy 0.863
train loss 0.559 valid loss 0.564 accuracy 0.868
train loss 0.559 valid loss 0.563 accuracy 0.868
train loss 0.558 valid loss 0.563 accuracy 0.867
train loss 0.557 valid loss 0.563 accuracy 0.865
train loss 0.557 valid loss 0.563 accuracy 0.865
train loss 0.557 valid loss 0.563 accuracy 0.866
train loss 0.556 valid loss 0.563 accuracy 0.868
train loss 0.556 valid loss 0.564 accuracy 0.869


In [263]:
#Testing
test = pd.read_csv('test_kaggle.csv')
test_ids = test['id']
test = test.merge(items_id_df, left_on='item_id', right_on='item_id').drop(columns=['id'])

In [256]:
users = torch.LongTensor(test.user_id.values)  
items = torch.LongTensor(test.item_id.values) 
context_feature_id = torch.LongTensor(test.context_feature_id.values)
item_feature_id = torch.LongTensor(test.item_feature_id.values)
y_hat_final = model(users, items, item_feature_id, context_feature_id)

In [277]:
kaggle_df = pd.DataFrame(list(y_hat_final.detach().numpy()), index=test_ids).rename({0: 'rating'}, axis='columns')

In [278]:
kaggle_df.to_csv('submission.csv')